In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

/home/arrow13/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen/'

In [8]:
var_pc = pc_frederik

In [9]:
cd $var_pc'/Messdaten/'

[Errno 2] No such file or directory: '/home/frederik/Dokumente/FP/Roentgen//Messdaten/'
/home/arrow13/Git/FP/Roentgen/Messdaten


In [10]:
data = np.loadtxt("Si111_40_30.xy").T

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdefraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [24]:
intervall_voigt_1 = np.arange(np.where(data[0] > 25.5)[0][0], np.where(data[0] > 25.8)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [25]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])

In [38]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/err_1)

In [39]:
out_1.plot()
plt.show()

In [40]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 17
    # data points      = 71
    # variables        = 3
    chi-square         = 3871.373
    reduced chi-square = 56.932
[[Variables]]
    voigt_1sigma:       0.01192755 +/- 0.000210 (1.76%) (init= 0.01357791)
    voigt_1center:      25.6459249 +/- 0.000353 (0.00%) (init= 25.64617)
    voigt_1amplitude:   1172.19255 +/- 17.18171 (1.47%) (init= 1214.115)
    voigt_1fwhm:        0.04295483 +/- 0.000756 (1.76%)  == '3.6013100*voigt_1sigma'
    voigt_1gamma:       0.01192755 +/- 0.000210 (1.76%)  == 'voigt_1sigma'
[[Correlations]] (unreported correlations are <  0.100)


### Mulit-Voigt

In [82]:
intervall_voigt_2 = np.arange(np.where(data[0] > 28)[0][0], np.where(data[0] > 29)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [86]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))

In [102]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2*10))

In [111]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2*10,fmt='. ')
plt.show()

In [104]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 36
    # data points      = 239
    # variables        = 6
    chi-square         = 244.869
    reduced chi-square = 1.051
[[Variables]]
    voigt_2sigma:       0.01536734 +/- 0.000330 (2.15%) (init= 0.0149357)
    voigt_2center:      28.4338329 +/- 0.000527 (0.00%) (init= 28.43486)
    voigt_2amplitude:   2653.25995 +/- 47.20420 (1.78%) (init= 1946.006)
    voigt_2fwhm:        0.05534258 +/- 0.001188 (2.15%)  == '3.6013100*voigt_2sigma'
    voigt_2gamma:       0.01536734 +/- 0.000330 (2.15%)  == 'voigt_2sigma'
    voigt_3amplitude:   1591.96591 +/- 41.47299 (2.61%) (init= 1136.358)
    voigt_3center:      28.5084768 +/- 0.000539 (0.00%) (init= 28.50588)
    voigt_3sigma:       0.01140284 +/- 0.000339 (2.98%) (init= 0.01222012)
    voigt_3fwhm:        0.04106517 +/- 0.001224 (2.98%)  == '3.6013100*voigt_3sigma'
    voigt_3gamma:       0.01140284 +/- 0.000339 (2.98